In [26]:
%pip install feedparser

In [27]:
from IPython import get_ipython
from IPython.display import display
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import feedparser
import pandas as pd
import time
import re
import concurrent.futures


In [28]:
# %%
%pip install feedparser
# %%
from IPython import get_ipython
from IPython.display import display
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import feedparser
import pandas as pd
import time
import re
import concurrent.futures

# %%
def parse_tribun(soup, url, source):
    """
    Mengurai objek BeautifulSoup untuk website tribunnews.com.

    Args:
        soup (BeautifulSoup object): Objek BeautifulSoup dari halaman yang di-scrape.
        url (str): URL artikel.
        source (str): Nama sumber website.

    Returns:
        dict or None: Dictionary data artikel jika berhasil, None jika gagal.
    """
    try:
        judul_tag = soup.select_one("h1")
        if not judul_tag:
            print("  ⚠️ Tidak menemukan tag <h1> untuk tribunnews.com")
            return None

        isi_paragraf = soup.select(".side-article p")
        if not isi_paragraf:
            print("  ⚠️ Tidak menemukan isi artikel untuk tribunnews.com")
            return None

        judul = judul_tag.text.strip()
        isi = "\n".join([p.text.strip() for p in isi_paragraf if p.text.strip()])

        # Tanggal dan waktu
        tanggal_tag = soup.select_one("time") or soup.select_one(".date")
        tanggal_full = tanggal_tag.text.strip() if tanggal_tag else "Tidak ditemukan"
        match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})[ ,]*(\d{1,2}:\d{2})?", tanggal_full)
        date = match.group(1) if match else tanggal_full
        time_ = match.group(2) if match and match.group(2) else ""

        # Breadcrumb: kategori dan sub-kategori
        breadcrumb = soup.select(".breadcrumb li a")
        category = breadcrumb[1].text.strip() if len(breadcrumb) > 1 else ""
        sub_category = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else ""

        return {
            "title": judul,
            "source": source,
            "date": date,
            "time": time_,
            "category": category,
            "sub-category": sub_category,
            "content": isi,
            "url": url,
        }

    except Exception as e:
        print(f"  ❌ Gagal mengurai tribunnews.com {url}: {e}")
        return None

# %%
def parse_sindonews(soup, url, source):
    """
    Mengurai objek BeautifulSoup untuk website sindonews.com.

    Args:
        soup (BeautifulSoup object): Objek BeautifulSoup dari halaman yang di-scrape.
        url (str): URL artikel.
        source (str): Nama sumber website.

    Returns:
        dict or None: Dictionary data artikel jika berhasil, None jika gagal.
    """
    try:
        judul_tag = soup.select_one(".detail-title")
        if not judul_tag:
             print("  ⚠️ Tidak menemukan .detail-title untuk sindonews.com")
             return None

        tanggal_tag = soup.select_one(".detail-date-artikel")
        if not tanggal_tag:
            print("  ⚠️ Tidak menemukan .detail-date-artikel untuk sindonews.com")
            # Masih bisa lanjut tanpa tanggal/waktu, beri nilai default
            tanggal_full = "Tidak ditemukan"
        else:
            tanggal_full = tanggal_tag.text.strip()


        isi_div = soup.select_one("html body div.row-content.mb40 div.left article div#detail-desc.detail-desc")
        if not isi_div:
            print("  ⚠️ Tidak menemukan #detail-desc untuk sindonews.com")
            return None # Konten adalah elemen penting

        judul = judul_tag.text.strip()

        # Mengambil teks dari semua paragraf di dalam div #detail-desc
        isi = "\n".join([p.text.strip() for p in isi_div.find_all("p") if p.text.strip()])

        # Parsing tanggal dan waktu
        match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})[ ,]*(\d{1,2}:\d{2})?", tanggal_full)
        date = match.group(1) if match else tanggal_full
        time_ = match.group(2) if match and match.group(2) else ""

        # Kategori dan Sub-kategori (Perlu dicari selector yang sesuai di Sindonews)
        # Untuk contoh, kita bisa coba cari di breadcrumb atau meta tag
        # Jika sulit ditemukan selector umum, mungkin perlu diatur manual atau dibiarkan kosong.
        # Contoh: mencari di breadcrumb
        breadcrumb = soup.select(".breadcrumb li a") # Coba selector umum, mungkin beda
        category = breadcrumb[1].text.strip() if len(breadcrumb) > 1 else ""
        sub_category = breadcrumb[2].text.strip() if len(breadcrumb) > 2 else ""

        # Jika breadcrumb di atas tidak bekerja, bisa coba meta tag atau bagian lain di HTML
        # meta_category = soup.select_one('meta[property="article:section"]')
        # category = meta_category['content'] if meta_category and 'content' in meta_category.attrs else ""

        return {
            "title": judul,
            "source": source,
            "date": date,
            "time": time_,
            "category": category, # Mungkin perlu disesuaikan
            "sub-category": sub_category, # Mungkin perlu disesuaikan
            "content": isi,
            "url": url,
        }

    except Exception as e:
        print(f"  ❌ Gagal mengurai sindonews.com {url}: {e}")
        return None

# %%
def scrape_berita(url, source):
    """
    Mengambil HTML dan memanggil fungsi parsing yang sesuai berdasarkan sumber.

    Args:
        url (str): URL artikel.
        source (str): Nama sumber website.

    Returns:
        dict or None: Dictionary data artikel jika berhasil di-scrape dan diurai, None jika gagal.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com/",
        "Accept-Language": "id,en-US;q=0.9,en;q=0.8",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    }
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        if resp.status_code != 200:
            print(f"  ⚠️ Status code: {resp.status_code} for {url}")
            return None
        soup = BeautifulSoup(resp.content, "html.parser")

        # Pilih fungsi parsing berdasarkan sumber
        if "tribunnews.com" in source:
            return parse_tribun(soup, url, source)
        elif "sindonews.com" in source:
            return parse_sindonews(soup, url, source)
        # Tambahkan elif untuk website lain di sini
        # elif "namadomainlain.com" in source:
        #    return parse_namadomainlain(soup, url, source)
        else:
            print(f"  ⚠️ Sumber tidak dikenali untuk parsing: {source}")
            return None

    except Exception as e:
        print(f"  ❌ Gagal mengambil atau memproses HTML dari {url}: {e}")
        return None


# %%
def full_rss_scrape(list_of_rss_urls, output_csv="berita.csv", max_articles=1000):
    """
    Mengambil link artikel dari daftar URL RSS, kemudian melakukan scraping,
    dan menyimpan hasilnya ke file CSV.

    Args:
        list_of_rss_urls (list): Daftar URL RSS yang akan diambil link-nya.
        output_csv (str): Nama file CSV untuk menyimpan hasil scraping.
        max_articles (int): Jumlah maksimum artikel yang akan di-scrape dari link RSS.

    Returns:
        pandas.DataFrame: DataFrame yang berisi data artikel yang berhasil di-scrape.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://www.google.com/",
    }

    urls = set()
    print("➡️ Mengumpulkan link dari RSS...")
    for rss_url in list_of_rss_urls:
        try:
            print(f"  Mengambil dari: {rss_url}")
            resp = requests.get(rss_url, headers=headers, timeout=10)
            feed = feedparser.parse(resp.text)
            for entry in feed.entries:
                # Hanya tambahkan jika link tidak kosong
                if entry.link:
                  urls.add(entry.link)
            time.sleep(1) # Memberi jeda agar tidak terlalu cepat
        except Exception as e:
            print(f"  ❌ Gagal ambil dari {rss_url}: {e}")

    # Ambil sejumlah URL sesuai parameter max_articles
    urls_to_scrape = list(urls)[:max_articles]
    print(f"✅ Total link unik yang berhasil dikumpulkan: {len(urls)}")
    print(f"➡️ Akan men-scrape {len(urls_to_scrape)} artikel...")

    print("➡️ Memulai proses scraping artikel...")
    def scrape_wrapper(args):
        url, source = args
        print(f"  Scraping: {url} | Source: {source}")
        # Memanggil fungsi scrape_berita yang sudah diperbarui
        return scrape_berita(url, source)

    berita_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Menggunakan urlparse untuk mendapatkan source (nama domain)
        args_list = [(url, urlparse(url).netloc) for url in urls_to_scrape]
        results = list(executor.map(scrape_wrapper, args_list))
        for data in results:
            if data:
                berita_list.append(data)

    df = pd.DataFrame(berita_list, columns=[
        "title", "source", "date", "time", "category", "sub-category", "content", "url"
    ])

    # Simpan hasil ke CSV
    print(f"➡️ Menyimpan hasil ke {output_csv}...")
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV disimpan! Jumlah artikel berhasil di-scrape: {len(df)}")

    return df

In [29]:
# Contoh penggunaan fungsi full_rss_scrape:

# Ambil daftar link RSS dari file CSV (misalnya hanya untuk 'tribun')
dataset = pd.read_csv('https://raw.githubusercontent.com/gikirima/indonews-scrapper/refs/heads/main/link_scrapping.csv')
tribun_rss_links = dataset[dataset['website'] == 'tribun']['link'].tolist()
sindonews_rss_links = dataset[dataset['website'] == 'sindonews']['link'].tolist()
liputan6_rss_links = dataset[dataset['website'] == 'liputan6']['link'].tolist()
detik_rss_links = dataset[dataset['website'] == 'detik']['link'].tolist()
kapanlagi_rss_links = dataset[dataset['website'] == 'kapanlagi']['link'].tolist()
fimela_rss_links = dataset[dataset['website'] == 'fimela']['link'].tolist()
okezone_rss_links = dataset[dataset['website'] == 'okezone']['link'].tolist()
posmetro_rss_links = dataset[dataset['website'] == 'posmetro']['link'].tolist()
kompas_rss_links = dataset[dataset['website'] == 'kompas']['link'].tolist()
republika_rss_links = dataset[dataset['website'] == 'republika']['link'].tolist()
tempo_rss_links = dataset[dataset['website'] == 'tempo']['link'].tolist()

In [30]:
# Panggil fungsi full_rss_scrape dengan daftar link RSS
# Anda bisa menyesuaikan nama file output CSV dan jumlah maksimum artikel
df_tribun = full_rss_scrape(tribun_rss_links, output_csv="berita_tribun.csv", max_articles=500)

# Jika Anda ingin melihat DataFrame hasilnya:
display(df_tribun.head())

➡️ Mengumpulkan link dari RSS...
  Mengambil dari: https://www.tribunnews.com/rss
  Mengambil dari: https://medan.tribunnews.com/rss
  Mengambil dari: https://jabar.tribunnews.com/rss
  Mengambil dari: https://surabaya.tribunnews.com/rss
  Mengambil dari: http://jabar.tribunnews.com/rss
  Mengambil dari: https://wartakota.tribunnews.com/rss
  Mengambil dari: https://jogja.tribunnews.com/rss
  Mengambil dari: http://jateng.tribunnews.com/rss
  Mengambil dari: http://aceh.tribunnews.com/rss
  Mengambil dari: https://www.kompas.com/rss
  Mengambil dari: https://regional.kompas.com/rss
✅ Total link unik yang berhasil dikumpulkan: 0
➡️ Akan men-scrape 0 artikel...
➡️ Memulai proses scraping artikel...
➡️ Menyimpan hasil ke berita_tribun.csv...
✅ CSV disimpan! Jumlah artikel berhasil di-scrape: 0


,title,source,date,time,category,sub-category,content,url


In [31]:
df_sindonews = full_rss_scrape(sindonews_rss_links, output_csv="berita_sindonews.csv", max_articles=500)
# df_liputan6 = full_rss_scrape(liputan6_rss_links, output_csv="berita_liputan6.csv", max_articles=500)
# df_detik = full_rss_scrape(detik_rss_links, output_csv="berita_detik", max_articles=500)
# df_kapanlagi = full_rss_scrape(kapanlagi_rss_links, output_csv="berita_kapanlagi", max_articles=500)
# df_fimela = full_rss_scrape(fimela_rss_links, output_csv="berita_fimela", max_articles=500)
# df_okezone = full_rss_scrape(okezone_rss_links, output_csv="berita_okezone", max_articles=500)
# df_posmetro = full_rss_scrape(posmetro_rss_links, output_csv="berita_posmetro", max_articles=500)
# df_kompas = full_rss_scrape(kompas_rss_links, output_csv="berita_kompas", max_articles=500)
# df_republika = full_rss_scrape(republika_rss_links, output_csv="berita_republika", max_articles=500)
# df_tempo = full_rss_scrape(tempo_rss_links, output_csv="berita_tempo", max_articles=500)

➡️ Mengumpulkan link dari RSS...
  Mengambil dari: https://www.sindonews.com/feed
  Mengambil dari: https://nasional.sindonews.com/rss
  Mengambil dari: https://international.sindonews.com/rss
  Mengambil dari: https://daerah.sindonews.com/rss
  Mengambil dari: https://lifestyle.sindonews.com/rss
  Mengambil dari: https://sports.sindonews.com/rss
  Mengambil dari: https://ekbis.sindonews.com/rss
  Mengambil dari: https://kalam.sindonews.com/rss
  Mengambil dari: https://tekno.sindonews.com/rss
  Mengambil dari: https://otomotif.sindonews.com/rss
  Mengambil dari: https://edukasi.sindonews.com/rss
✅ Total link unik yang berhasil dikumpulkan: 306
➡️ Akan men-scrape 306 artikel...
➡️ Memulai proses scraping artikel...
  Scraping: https://international.sindonews.com/read/1578365/42/trump-dan-elon-musk-bertikai-sengit-para-pegawai-doge-khawatir-akan-dipecat-doge-1749549961 | Source: international.sindonews.com
  Scraping: https://international.sindonews.com/read/1578093/41/memalukan-inggris